In [ ]:
import zipfile
path_to_zip_file = 'Abstracts.zip'    #Path to zip file
directory_to_extract_to = 'Extracted'  #Directory name
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)           #extracting files

In [ ]:
import os
import codecs
import string
from nltk.stem import PorterStemmer
import re
docs_list = []
doc_id_list=[]
content=''                
for filename in os.listdir('Extracted/Abstracts'):                      
    if filename.endswith('.txt'):
      pattern = '(\d*).txt'                                               #removing .txt from file names using regular expression
      docid=re.findall(pattern,filename)
      doc_id = docid[0]                                                   
      doc_id_list.append(doc_id)                             #storing doc ids in doc_list as files are not processed sequentially by os, so to sync later
      with codecs.open(os.path.join('Extracted/Abstracts', filename), 'r', encoding='utf-8',errors='ignore') as fdata:
        doc = fdata.read()                        #reading data 
        content= content+doc                      #to form dictionary
        docs_list.append(doc)                      #to form posting lists

In [ ]:
def tokenize(content):
  for char in string.punctuation:
    content2 = content.replace(char,' ')                  #removing punctuations
  tokens = content2.split()                               #splitting into tokens
  tokens.sort()
  tokens2 = [i.lower() for i in tokens]                   #case normalization
  f = open('Stopword-List.txt')
  stop_words = f.read()
  stop_words_list = stop_words.split()
  tokens3 = tokens2.copy()
  tokens3.sort()
  
  #print(tokens3)

  for i in stop_words_list:
    if i in tokens3:
      tokens3 = [x for x in tokens3 if i!=x]                            #removing stop words
  tokens4 = tokens3.copy()
  tokens4 = list(set(tokens3))                                          #removing duplicates
  tokens4.sort()
  tokens5 = tokens4.copy()
  tokens6=[]
  ps = PorterStemmer()
  for i in tokens5:
    tokens6.append(ps.stem(i))
  
  tokens7 = [i for i in tokens6 if not any(c.isdigit() for c in i)]           #removing digits
  tokens7.sort()                                                              #sorting alphabetically 
  tokens8 = [i for i in tokens7 if len(i)>1]                                  #removing single character strings
  #print(tokens7)
  return tokens8

In [ ]:
def tokenize_doc(content):                          #same function as above except no sorting and no removing of duplicates for positional index 
  for char in string.punctuation:
    content2 = content.replace(char,' ')
  tokens = content2.split()
  #tokens.sort()
  tokens2 = [i.lower() for i in tokens]
  f = open('Stopword-List.txt')
  stop_words = f.read()
  stop_words_list = stop_words.split()
  tokens3 = tokens2.copy()
  #tokens3.sort()
  
  #print(tokens3)

  for i in stop_words_list:
    if i in tokens3:
      tokens3 = [x for x in tokens3 if i!=x]
  tokens4 = tokens3.copy()
  #tokens4 = list(set(tokens3))
  #tokens4.sort()
  tokens5 = tokens4.copy()
  tokens6=[]
  ps = PorterStemmer()
  for i in tokens5:
    tokens6.append(ps.stem(i))
    tokens7 = [i for i in tokens6 if not any(c.isdigit() for c in i)]
  #tokens7.sort()
  tokens8 = [i for i in tokens7 if len(i)>1]
  #print(tokens7)
  return tokens8

In [ ]:
doc_tokens=[]                                     #maintaining list of doc_tokens for indexes      
for doc in docs_list:
  tok = tokenize_doc(doc)
  doc_tokens.append(tok)

In [ ]:
def remove_punctuation(content):                #seprate removing pucntuation to remove punctuation from whole content
  content2=content
  for char in string.punctuation:
    content2 = content2.replace(char,' ')
  return content2

In [ ]:
content_ret= remove_punctuation(content)         
tokens = tokenize(content_ret)                  #tokenizing the content (whole collection string)
for token in tokens[0:10]:                      #checking first few tokens
  print(token)

aal
aapl
abbrevi
abc
abdomin
abduct
abil
abil
abl
abnorm


In [ ]:
d = {k:[] for k in tokens}    #adding keys to dictionary

In [ ]:
for token in tokens:                  #adding posting list to dictionary
  i=0                                 #used to sync the document ids
  for doc_token in doc_tokens:
    if(token in doc_token):
       d[token].append(doc_id_list[i])
    i=i+1

In [ ]:
temp=d.copy()
for key,value in temp.items():       #removing duplicates from dictionary 
  if(len(value)==0):
    del d[key]

In [ ]:
for k,v in d.items():       #removing duplicates from docs and sorting them in ascending order (duplicates because same list for positional and inverted)
  v = list(set(v))
  v.sort()
  d[k]=v

In [ ]:
import json
with open("inverted_index2.json", "w") as outfile:      #storing inversted index
    json.dump(d, outfile)

In [ ]:
pi = {k:[] for k in tokens}      #adding keys to positional index

In [ ]:
for token in tokens:                    #creating positional index
  i=0
  for doc_token in doc_tokens:
    if(token in doc_token):                     #checking the occurance in document
       doc_id= doc_id_list[i]                   #getting doc id same as inverted index
       pos=doc_tokens[i].index(token)            #getting position by fetching index of token in tokenized documents
       x = (doc_id,pos)                     #doc id, position tuple
       pi[token].append(x)                       #appending tuples into the index           
    i=i+1

In [ ]:
temp=pi.copy()                      #removing duplicates from dictionary
for key,value in temp.items():
  if(len(value)==0):
    del pi[key]

In [ ]:
import json
with open("positional_index2.json", "w") as outfile:        #storing positional index
    json.dump(pi, outfile)

In [ ]:
def proximity_queries(query_terms,distance,pi,d):
                                                      
  result_set= []
  retrieved_lists = []
  j=0
  ps= PorterStemmer()
  qt=[]
  for i in query_terms:
    i = ps.stem(i)
    qt.append(i)

  rs={k:[] for k in qt}
  rs2={k:[] for k in qt}
  for i in qt:
    rs[i].append(pi[i])
    retrieved_lists.append(d[i])
  result_set = retrieved_lists[0]

  for i in range(len(query_terms)-1):
    result_set = list(set(result_set) & set(retrieved_lists[i+1]))

  x = ()
  for k,v in rs.items():
    for i in v:
      for j in i:
        for m in result_set:
          if(m==j[0]):
            rs2[k].append((j[0],j[1]))
  

  for k,v in rs2.items():    
    v = list(set(v))
    v.sort()
    rs2[k]=v

  n = len(rs2[qt[0]])
  n2 = len(rs2)


  final_set=[]
  for i in range(0,n):
    dist=[]
    for j in range(0,n2):
      dist.append(rs2[qt[j]][i][1])
    for j in range(0,n2):
      if(abs(dist[0]-dist[1])<=distance):
        final_set.append(rs2[qt[j]][i][0])
  final_set = list(set(final_set))
  return final_set

In [ ]:
query_terms = ['neural','information']
proximity_queries(query_terms,3,pi,d)

['26']

In [ ]:
def simple_boolean_query(query_terms,d,operations):       #function to resolve simple boolean query
  stemmed_query=[]                                       
  retrieved_lists = []                    
  ps= PorterStemmer()
  for i in query_terms:
    if i[-1]=='!':
      temp=i
      i=i[:-1]
      s=ps.stem(i)
      rl= d[s]
      v = [str(i) for i in range(1,448) if str(i) not in rl]     #taking compliment if term is with NOT
      retrieved_lists.append(v)
      continue
    s=ps.stem(i)   
                                        #stemming the query
    retrieved_lists.append(d[s])                    #adding doc ids in retrieved list
  result_set = retrieved_lists[0]                   #adding first posting in reseult set
  j=1
  for i in operations:
    if i==0:
      result_set = list(set(result_set) & set(retrieved_lists[j]))      #checking if operation is and/or and applying te operation with next posting
    elif i==1:
      result_set = list(set(set(result_set) | set(retrieved_lists[j])))

    j=j+1
  return (result_set)

In [ ]:
def query_preprocessing(query,d):
  breaking = query.split()         
  query_terms=[]
  operations = []
  flag=False
  for i in breaking:
    if(i=='AND'):
      operations.append(0)              #using operations list to specify the operation
    elif (i=='OR'):
      operations.append(1)
    elif (i=='NOT'):    
      flag=True
    else:
      if flag==True:
        i = i+'!'                        #adding ! to mark it as NOT to compliment later
      flag=False
      query_terms.append(i)
  rs=simple_boolean_query(query_terms,d,operations)
  #rs.sort()

  return rs

In [ ]:
query = 'image AND NOT learning AND machine'
query_preprocessing(query,d)

['371', '102', '126', '125']

In [ ]:
def proximity_queries_processing(query,d,pi):
  query_terms = []
  pattern= '\d'
  match=re.findall(pattern,query)
  distance = int(match[0])
  for i in query.split():
   query_terms.append(i)
  query_terms.pop()
  return (proximity_queries(query_terms,distance,pi,d))

In [ ]:
query = 'feature track /5'
print(proximity_queries_processing(query,d,pi))

['212', '13']


In [ ]:
def phrase_query(phrase,d,pi):
  terms = phrase.split()
  n=len(terms)
  operations=[]
  for i in range(n-1):
    operations.append(0)
  rs = simple_boolean_query(terms,d,operations)       #initially applying simple boolean model
  
  if(len(rs)>10):                                     #if result set more than 10 then using proximity query
    rs=proximity_queries(terms,10,pi,d)
  return rs

In [ ]:
def classify_query(query,d,pi):
  if('NOT' in query or 'AND' in query or 'OR' in query):
    print(query_preprocessing(query,d))

  elif query[-1].isdigit():
    print(proximity_queries_processing(query,d,pi))

  else:
    print(phrase_query(query,d,pi))


['212', '13']


In [236]:
query = 'image AND restoration'
classify_query(query,d,pi)

['359', '375']


In [237]:
query = 'time AND series OR classification'
classify_query(query,d,pi)

['75', '107', '9', '438', '66', '445', '280', '338', '384', '234', '51', '46', '235', '158', '286', '302', '256', '77', '6', '173', '277', '84', '259', '182', '377', '261', '442', '295', '345', '341', '167', '283', '353', '140', '60', '55', '193', '94', '371', '128', '40', '56', '175', '120', '198', '33', '10', '317', '354', '375', '143', '95', '123', '305', '125', '238', '71', '421', '228', '174', '239', '395', '245', '252', '220', '126', '171', '357', '64', '272', '397', '287', '24', '215', '334', '164', '22', '386', '58', '85', '249', '247', '176', '122', '59', '268', '369', '99', '38', '208', '303', '76', '210', '427', '321', '165', '177', '439', '240', '404', '111', '385', '432', '113', '187', '405', '236', '237', '348', '299', '328', '45', '425', '202', '378', '265', '350', '168', '16', '273', '424', '63', '34', '352', '4', '313', '73', '43', '316', '387', '54', '229', '67', '197', '106']


In [238]:
query = 'feature track /5'
classify_query(query,d,pi)

['212', '13']


In [239]:
query=input()
classify_query(query,d,pi)

image processing
['247', '162', '366', '358', '352']
